# Stackleberg equilibrium

This notebook is used to find the stackleberg equilibrium for a game in extensive form translated into normal-form.

In [103]:
from amplpy import AMPL
import pandas as pd

In [104]:
def data_preprocesing(players, actions, utility_matrices):
    actions_encoded = {player: [i for i in range(len(actions[player]))] for player in players}
    utilities_df = {player: pd.DataFrame(utility_matrices[player], index=actions_encoded['leader'], columns=actions_encoded['follower']) for player in players}
    
    return utilities_df, actions_encoded

def stackleberg_equilibrium(players, actions, utilities):
    utilities_df, actions_encoded = data_preprocesing(players, actions, utilities)
    
    ampl = AMPL()
    ampl.read('normal_stackleberg.mod')
    
    ampl.set['Al'] = actions_encoded['leader']
    ampl.set['Af'] = actions_encoded['follower']
    
    ampl.param['Ul'] = utilities_df['leader']
    ampl.param['Uf'] = utilities_df['follower']
    
    optimal = {}
    for action_follower in actions_encoded['follower']:
        ampl.param['Af_bar'] = action_follower
        ampl.solve(solver='gurobi')
        assert ampl.solve_result == "solved"
        
        obj = ampl.getObjective('obj').value()
        sl = ampl.getVariable('sl').getValues().toPandas()
        
        if obj > optimal.get('obj', -float('inf')):
            optimal = {'Af_bar': actions['follower'][action_follower], 'obj': obj, 'sl': sl}
    
    ampl.close()
    
    return optimal
    

In [105]:
# Game
players = ['leader', 'follower']

A_1 = ["t1", "t2", "t3", "t4"]
A_2 = ["t1", "t2", "t3", "t4"]

I = {'leader': A_1, 'follower': A_2}

U1 = [[1, 0.5, 0.5, 0.5],
    [0.5, 1, 0.5, 0.5],
    [0.5, 0.5, 1, 0.5],
    [0.5, 0.5, 0.5, 1]]

U2 = [[0, 0.4, 0.3, 0.3],
    [0.6, 0, 0.4, 0.3],
    [0.7, 0.6, 0, 0.4],
    [0.7, 0.7, 0.6, 0]]

U = {'leader': U1, 'follower': U2}

optimal = stackleberg_equilibrium(players, I, U)

print("\nRESULTS:")
print("Optimal action for follower: ", optimal['Af_bar'])
print("Optimal strategy for leader: ")
for index, row in optimal['sl'].iterrows():
    print(f" {I['leader'][index]} -> {row['sl.val']}")
print("Optimal objective value: ", optimal['obj'])

Gurobi 9.5.1: optimal solution; objective 0.7598039216
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.662745098
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.562745098
Gurobi 9.5.1: optimal solution; objective 0.5147058824

RESULTS:
Optimal action for follower:  t1
Optimal strategy for leader: 
 t1 -> 0.5196078431372549
 t2 -> 0.3254901960784313
 t3 -> 0.1254901960784314
 t4 -> 0.02941176470588236
Optimal objective value:  0.7598039215686274
